In [9]:
import numpy as np
from miditoolkit.midi import parser as mid_parser  
from miditoolkit.midi import containers as ct
PAD=[2,48,86,192,97,3]
EOS=[4,50,88,194,99,5]
ABS=[5,51,89,195,100,6]

In [10]:
def token2mid(page, out_path):
    # meta data
    out = mid_parser.MidiFile()
    out.ticks_per_beat = 480

    # Assume piano for simplicity, but can get from token[4]
    out.instruments = [ct.Instrument(
        program=0, is_drum=False, name='reduction')]

    # First Time Signature
    ts = int(page[0][5]+2)
    last_ts = ts
    current_beat = -ts*480
    out.time_signature_changes.append(ct.TimeSignature(ts, 4, 0))

    for idx, n in enumerate(page):
        # Stop if end or padding starts
        if (n == EOS).all() or (n == PAD).all():
            break

        # Time Signature for THIS note
        ts = int(page[idx][5]+2)
        # Bar moves forward
        if n[0] == 0 or (n[:-1] == ABS[:-1]).all():
            current_beat += last_ts*480
        # Update new Time Signature if any
        if ts != last_ts:
            last_ts = ts
            out.time_signature_changes.append(ct.TimeSignature(ts, 4, current_beat))

        # Add THIS note
        if (n[:-1] != ABS[:-1]).all():
            out.instruments[0].notes.append(ct.Note(
                start=int(
                    current_beat+n[1]*480/12),
                end=int(
                    current_beat +
                    (n[1]+n[3]+1)*(480/12)),
                pitch=n[2]+22,
                velocity=90))
    out.dump(out_path)


In [11]:
# Token name
npy_file = "out/.npy"
mid_file = "out/out.mid"

# Load token for the FIRST song
token = np.load(npy_file)[0]
token2mid(token, mid_file)

We support up to sixteenth note. For a thirty-second note, it is 1.5 tick.

If you want to support thirty-second note or higher (rare), we have to upgrade 48 -> 84 beats. More ticks may add burdens to training.